In [ ]:
import scipy
import xarray as xr
import numpy as np
from netCDF4 import Dataset
import os, shutil
from typing import List
import xesmf as xe

Define some helper functions for creating masks.
Generate the required output file based on a default temperature file.
The input grid for oxygen is 1/4 degrees while the output grid is 1/12th degrees.

In [ ]:
def convert_CF_compliant_file_to_output_file(dst_grd_file, var_name="o2"):
    out_file="climatology_glorys12v1_{}.nc".format(var_name)
    print("Output climatology file will be named {}".format(out_file))
    if os.path.exists(out_file): os.remove(out_file)
    shutil.copy2(dst_grd_file, out_file)
    dst_ds = Dataset(out_file, "r+")

    dst_ds.renameVariable("thetao", var_name)

    dst_ds[var_name].long_name = "Dissolved Oxygen"
    dst_ds[var_name].standard_name="oxygen"
    dst_ds[var_name].units="ml/l"
    dst_ds[var_name].unit_long="Dissolved Oxygen"
    dst_ds.close()
    return out_file

def _create_mask(ds:xr.Dataset, threshold):

    """ creates a SCRIP-compatible mask DataArray made of 0 (masked) and 1 (unmasked)

    ds : xr.Dataset()

    mask_in, mask_out: dict
      dict{variable: tuple(threshold, direction)}
      direction is either "smaller", "equal", "greater"
      NaNs will always be masked. If this is the only thing to mask, use np.nan as the threshold
      example: {"sftlf": (0, "equal")}

    :return mask : xr.DataArray()
    """

    v = list(threshold.keys())[0]
    t = threshold[v]

    if "time" in ds:
        ds = ds.isel(time=0)

    if t[1] in ["equal"]:
        mask = xr.where((xr.ufuncs.isnan(ds[v])) | (ds[v] == t[0]), 0, 1)
    elif t[1] in ["smaller"]:
        mask = xr.where((xr.ufuncs.isnan(ds[v])) | (ds[v] <= t[0]), 0, 1)
    elif t[1] in ["greater"]:
        mask = xr.where((xr.ufuncs.isnan(ds[v])) | (ds[v] >= t[0]), 0, 1)
    else:
        logging.error(t[1] + " not recognized.")
        raise ValueError

    return mask

def add_matrix_nans(r):
    """Deal with wrong boundary"""
    X = r.weights
    M = scipy.sparse.csr_matrix(X)
    num_nonzeros = np.diff(M.indptr)
    M[num_nonzeros == 0, 0] = np.NaN
    r.weights = scipy.sparse.coo_matrix(M)

    return r

Read the source and destination grids and create datasets.
Convert the longitudes from -180-180 to 0-360 (required by xesmf and esmf)
Convert the coordinates names to lon and lat as required by xesmf

In [ ]:
src_grd_file="src_grid.nc"
dst_grd_file="dst_grid.nc"

out_file = convert_CF_compliant_file_to_output_file(dst_grd_file, var_name="o2")

climatology_src_ds = xr.open_dataset(src_grd_file)
climatology_src_ds_360 = climatology_src_ds.assign_coords(longitude=(((climatology_src_ds.longitude + 180) % 360) + 180)).sortby("longitude")
climatology_src_ds_360_trans = climatology_src_ds_360.transpose('time', 'depth', 'latitude', 'longitude')
src_ds=climatology_src_ds_360_trans.rename({"longitude":"lon","latitude":"lat"})

climatology_dst_ds = xr.open_dataset(out_file)
climatology_dst_ds_360 = climatology_dst_ds.assign_coords(longitude=(((climatology_dst_ds.longitude + 180) % 360) + 180)).sortby("longitude")
climatology_dst_ds_360_trans = climatology_dst_ds_360.transpose('time', 'depth', 'latitude', 'longitude')
dst_ds=climatology_dst_ds_360_trans.rename({"longitude":"lon","latitude":"lat"})

regridder = xe.Regridder(src_ds, dst_ds, 'bilinear', extrap='inverse_dist', extrap_num_pnts=3, extrap_exp=10)
regridder = add_matrix_nans(regridder)

# prepare the masks
src_ds["mask"] = _create_mask(src_ds, {"o2": (np.nan, "equal")})
dst_ds["mask"] = _create_mask(dst_ds, {"o2": (np.nan, "equal")})

da_out = regridder(src_ds["o2"], keep_attrs=True).to_dataset()
da_out.to_netcdf("test2.nc")
print("Finished interpolating and extrapolating")

